In [10]:
from requests import get
from pprint import PrettyPrinter

import pandas as pd

printer = PrettyPrinter()

base_url = "https://data.nba.net"
all_json = "/prod/v1/today.json" 

In [ ]:
# NBA Stats - JSON Endpoints
# http://data.nba.net/10s/prod/v1/2021/players.json > Player data for a specific year
# https://data.nba.net/10s/prod/v1/2021/teams.json > Team data for a specific year
# http://data.nba.net/data/10s/prod/v1/20180928/0011800001_boxscore.json > Box score data for a specific game (needs date and game id)
# https://data.nba.net/10s/prod/v1/20180929/scoreboard.json > Box scores for all games on a specific day (needs date)
# https://data.nba.net/10s/prod/v1/2021/schedule.json > Schedule data for a specific year

In [ ]:
# CODE FOR PULLING TEAM ROSTER DATA
# team_roster_endpoint = api.data_links['teamRoster']
# team_roster_endpoint = team_roster_endpoint.replace('{{teamUrlCode}}','raptors')

# test_data = get(base_url + team_roster_endpoint).json()
# print(len(test_data['league']['standard']['players']))

In [ ]:
# player_data_df = pd.DataFrame(api.player_data)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# player_data_df.loc[player_data_df['teamId'] == '1610612748']

In [41]:
class NBAStatsAPI:

    base_url = "https://data.nba.net"
    all_json = "/prod/v1/today.json"

    
    def __init__(self):
        # Dict with JSON endpoints
        self._data_links = self._get_all_data_links()
        # List of dicts with each dict containing data for a team
        self._team_data = self._get_team_data()
        # List of dicts with each dict containing data for a player
        self._player_data = self._get_player_data()
        # List of dicts with each dict containing data for a scheduled game
        self._schedule_data = self._get_schedule_data()
        # Date data was last refreshed
        self._date_last_refreshed = self._data_links['currentDate']


    @property
    def data_links(self):
        return self._data_links
    
    @property
    def team_data(self):
        return self._team_data

    @property
    def player_data(self):
        return self._player_data

    @property
    def schedule_data(self):
        return self._schedule_data

    def _get_all_data_links(self):
        data = get(NBAStatsAPI.base_url + NBAStatsAPI.all_json).json()
        links = data['links']

        return links

    def _get_team_data(self):
        team_data_endpoint = get(base_url + self.data_links['teams']).json()
        team_data = team_data_endpoint['league']['standard']
        team_data = list(filter(lambda x: x['isNBAFranchise'] == True, team_data))

        return team_data

    def _get_player_data(self):
        player_data_endpoint = get(base_url + self.data_links['leagueRosterPlayers']).json()
        player_data = player_data_endpoint['league']['standard']
        
        return player_data
    
    def _get_schedule_data(self):
        schedule_data_endpoint = get(base_url + self.data_links['leagueSchedule']).json()
        schedule_data = schedule_data_endpoint['league']['standard']

        return schedule_data

    def get_game_data(self, game_date: str, game_id: str):
        game_data_link = self.data_links['boxscore']
        game_data_link = game_data_link.replace('{{gameDate}}', f'{game_date}')
        game_data_link = game_data_link.replace('{{gameId}}', f'{game_id}')

        game_data_endpoint = get(base_url + game_data_link).json()
        game_data = game_data_endpoint['basicGameData']

        return game_data

    def __str__(self):
        return f"Data Last Refreshed: {self._date_last_refreshed}"

In [30]:
api.data_links

{'anchorDate': '20220408',
 'currentDate': '20220408',
 'calendar': '/prod/v1/calendar.json',
 'todayScoreboard': '/prod/v1/20220408/scoreboard.json',
 'currentScoreboard': '/prod/v1/20220408/scoreboard.json',
 'teams': '/prod/v2/2021/teams.json',
 'scoreboard': '/prod/v2/{{gameDate}}/scoreboard.json',
 'leagueRosterPlayers': '/prod/v1/2021/players.json',
 'allstarRoster': '/prod/v1/allstar/2018/AS_roster.json',
 'leagueRosterCoaches': '/prod/v1/2021/coaches.json',
 'leagueSchedule': '/prod/v1/2021/schedule.json',
 'leagueConfStandings': '/prod/v1/current/standings_conference.json',
 'leagueDivStandings': '/prod/v1/current/standings_division.json',
 'leagueUngroupedStandings': '/prod/v1/current/standings_all.json',
 'leagueMiniStandings': '/prod/v1/current/standings_all_no_sort_keys.json',
 'leagueTeamStatsLeaders': '/prod/v1/2021/team_stats_rankings.json',
 'leagueLastFiveGameTeamStats': '/prod/v1/2021/team_stats_last_five_games.json',
 'previewArticle': '/prod/v1/{{gameDate}}/{{gameI

In [43]:
# api = NBAStatsAPI()

# api.get_game_data('20211107', '0022100139').keys()
# test_data = get(base_url + '/prod/v1/20211107/0022100139_mini_boxscore.json').json()['basicGameData']
# api.schedule_data[204]


Data Last Refreshed: 20220408
